# 自由振動のモデル

吉田勝俊（宇都宮大学）

## 参考情報

- [Pythonで運動方程式を解く(odeint) - Qiita](https://qiita.com/binaryneutronstar/items/ad5efa27fd626826846f)
- [[Python] Numpyの参照、抽出、結合 - Qiita](https://qiita.com/supersaiakujin/items/d63c73bb7b5aac43898a)
- [[Python/matplotlib] FuncAnimationを理解して使う - Qiita](https://qiita.com/osanshouo/items/3c66781f41884694838b)

In [ ]:
import numpy as np                              #数値計算ライブラリ
from numpy.linalg import norm                   #ベクトルのノルム
from scipy.integrate import odeint              #常微分方程式ライブラリ
import matplotlib.pyplot as plt                 #描画ライブラリ
from matplotlib import rc                       #グラフ調整ライブラリ
#Colab用の設定（グラフィックのインライン表示）
%matplotlib inline
#Colab用の設定（TeX形式の高品位な数式）
rc('text', usetex=True)

## 自由振動系の運動方程式

In [ ]:
def EOM(x, t, m, c, k):
    '''
    自由振動系の運動方程式を表す関数（１階化したもの）
    '''
    x1, x2 = x
    dxdt = np.array([
        x2,
        (-c*x2 - k*x1)/m,
    ])
    
    return dxdt

## 数値解

In [ ]:
def Solve(m, c, k, x0, time):
    '''
    自由振動系の数値解を求める関数
    '''

    ###運動方程式を数値的に解く
    motion = odeint(
        EOM,            #運動方程式を表すユーザ関数
        x0,             #初期条件
        time,           #時間軸を表す数列
        args=(m, c, k)  #運動方程式の係数
    )
  
    return motion

### 数値解を求める

* パラメータ　$m=1$, $c=0.2$, $k=1$
* 初期条件　$x(0)=2$, $\dot x(0)=0$ （引っ張って止めた状態）

初期条件をベクトルで表す．2だけ引っ張って（変位＝２），停止（速度＝０）

In [ ]:
x0 = np.array([
    2,  #初期変位 
    0,  #初速度
])

print('x0 =', x0)

時間軸を表す等差数列はこんな感じ．

In [ ]:
ts = np.linspace(0,20,100) #時間軸を表す数列（0〜20秒を100等分）
print('time =', ts)

運動方程式の数値解はこんな感じ．（変位，速度）の２次元ベクトルが，時刻ごとに，縦に並んでいます．
これを*相軌道*と呼びました．

In [ ]:
xs = Solve(m=1, c=0.2, k=1, x0=x0, time=ts)
print('motion = ', xs)

## 数値解の可視化

### 数値解から成分を取り出す

In [ ]:
x1s = xs[:,0]   #数値解の変位成分
x2s = xs[:,1]   #数値解の速度成分

### 変位の振動波形 $(t,x_1)$

In [ ]:
plt.plot(ts, x1s)
plt.xlabel('$t$', fontsize=20)
plt.ylabel('$x_1(t)$', fontsize=20)

### 速度の振動波形 $(t,x_2)$

In [ ]:
plt.plot(ts, x2s)
plt.xlabel('$t$', fontsize=20)
plt.ylabel('$x_2(t)$', fontsize=20)

### 相軌道 $(x_1,x_2)$

In [ ]:
plt.plot(x1s, x2s)
plt.xlabel('$x_1(t)$', fontsize=20)
plt.ylabel('$x_2(t)$', fontsize=20)

### アニメーション

In [ ]:
from matplotlib.animation import FuncAnimation  #アニメーションライブラリ

In [ ]:
def animate_motion(x1s):
    '''
    自由振動系の運動をアニメーションする関数
    '''

    fig = plt.figure(               #キャンバスを設ける
#        figsize=plt.figaspect(3/2)  #縦横比 高さ / 幅
        figsize=(2,3)  #幅, 高さ 
    )  
    ax = fig.add_subplot(1, 1, 1)   #グラフ用紙を作る

    w = 0.3 #描画する四角形の幅
    h = 0.2 #描画する四角形の高さ

    def each_frame(ti):
        '''
        パラパラマンガの1コマを描く関数
        '''
        ax.cla() #グラフをクリア

        #ti番目の時刻における変位    
        x1 = x1s[ti]  
        points = np.array([ #四角形を一筆書きするための点列
            [-w/2, x1-h/2],
            [-w/2, x1+h/2],
            [ w/2, x1+h/2],
            [ w/2, x1-h/2],
            [-w/2, x1-h/2],
        ])

        pxs = points[:,0] #点列のX成分だけの数列
        pys = points[:,1] #点列のY成分だけの数列
        
        #時刻tiの各質点の描画
        ax.plot(pxs, pys, '-', color='C1')

        ax.set_ylabel('$x_1$', fontsize=16)
        ax.set_xlim(-2,2)
        ax.set_ylim(-3,3)
        ax.grid()

        fig.tight_layout()

    anim = FuncAnimation(
        fig, each_frame, 
        interval=80, frames=len(x1s)
    )

    rc('animation', html='jshtml')
    return anim

次のセルを実行して，現れた図形の再生ボタンをクリックすると，動画が動きます．

In [ ]:
animate_motion(x1s) #若干時間を要します

## 演習2.6

同じ処理を繰り返すので，以上の処理を一括したユーザー関数を作っておきます．

In [ ]:
def all_plot(m, c, k, x0, time):
    '''
        数値解を求めて，全ての表示方法で表示する関数
    '''
    
    ### 数値解を求める
    xs = Solve(m=m, c=c, k=k, x0=x0, time=time)
    
    x1s = xs[:,0]   #数値解の変位成分
    x2s = xs[:,1]   #数値解の速度成分
    
    ### 振動波形を描く
    plt.figure(figsize=(8,3)) #新しいグラフ用紙
    plt.subplot(1,3,1) #グラフを1行3列表示の1番目
    plt.plot(time, x1s) #変位の振動波形
    plt.xlabel('$t$', fontsize=14)
    plt.ylabel('$x_1(t)$', fontsize=14)
    
    plt.subplot(1,3,2) #グラフを1行3列表示の1番目
    plt.plot(time, x2s) #速度の振動波形
    plt.xlabel('$t$', fontsize=14)
    plt.ylabel('$x_2(t)$', fontsize=14)
    
    ### 相軌道を描く
    plt.subplot(1,3,3) #グラフを1行3列表示の3番目
    plt.plot(x1s, x2s)
    plt.xlabel('$x_1(t)$', fontsize=14)
    plt.ylabel('$x_2(t)$', fontsize=14)
    
    plt.tight_layout() #レイアウト調整
    plt.show() #強制出力

    ### アニメーションする
    print('Generating animation (Please wait) ... ', end='')
    plt.figure() #新しいグラフ用紙
    anim = animate_motion(x1s)
    display(anim)


共通の設定（初期条件，時間軸）

In [ ]:
# 初期条件を表すベクトル
x0 = np.array([
    2,  #初期変位 
    0,  #初速度
])

# 時間軸を表す等差数列
ts = np.linspace(0,40,200) #ちょっと長めに0〜40秒を200等分

* もっと長時間観察したいときは，`linspace`の`40`を増やせばできます！
* その際，`200`を据え置くと，アニメーションが早回しになります．（単位時間当りのコマ数が減る）

### (1) 単振動（一定振幅で振動する）

* `m`, `c`, `k` を変更して，再現してください．

In [ ]:
all_plot(m=1, c=0.2, k=1, x0=x0, time=ts)

### (2) 減衰振動（振動しながら x = 0 に収束する）

* `m`, `c`, `k` を変更して，再現してください．

In [ ]:
all_plot(m=1, c=0.2, k=1, x0=x0, time=ts)

### (3) 無周期減衰運動（振動しないで x = 0 に収束する）

* `m`, `c`, `k` を変更して，再現してください．

In [ ]:
all_plot(m=1, c=0.2, k=1, x0=x0, time=ts)